In [1]:
import django

In [2]:
from updates.models import Chirp

In [3]:
Chirp.objects.all()

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: my first chirp by jeff posted my first chirp>, <Chirp: Second by jeff posted Second>, <Chirp: Third by jeff posted Third>, <Chirp: Bob first by jeff posted Bob first>, <Chirp: Bob second by jeff posted Bob second>, <Chirp: Actual Bob first by bob posted Actual Bob first>, <Chirp: Actual Bob second by bob posted Actual Bob second>, <Chirp: Testin by jeff posted Testin>]

In [5]:
Chirp.objects.exclude(author__username = 'jeff')

[<Chirp: Actual Bob first by bob posted Actual Bob first>, <Chirp: Actual Bob second by bob posted Actual Bob second>]

In [6]:
Chirp.objects.filter(author__username = 'bob')

[<Chirp: Actual Bob first by bob posted Actual Bob first>, <Chirp: Actual Bob second by bob posted Actual Bob second>]

In [8]:
Chirp.objects.filter(author__username = 'jeff').exclude(title__icontains='Bob')

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: my first chirp by jeff posted my first chirp>, <Chirp: Second by jeff posted Second>, <Chirp: Third by jeff posted Third>, <Chirp: Testin by jeff posted Testin>]

# Querysets are unique
q1 and q2 are completely separate from each other

In [10]:
q1 = Chirp.objects.filter(author__username = 'jeff')
q2 = q1.exclude(title__icontains='Bob')

# Querysets are Lazy
Only 1 database query will be executed

In [11]:
q = Chirp.objects.filter(author__username = 'jeff')
q = q.exclude(title__icontains='Bob')

# Get
Get just 1 object.  This is the same as objects.filter(id=1)[0]

In [12]:
Chirp.objects.get(pk=1)

<Chirp: my first chirp by jeff posted my first chirp>

In [13]:
Chirp.objects.filter(id=1)[0]

<Chirp: my first chirp by jeff posted my first chirp>

#### Raises

In [14]:
Chirp.objects.get(pk=100)

DoesNotExist: Chirp matching query does not exist.

In [15]:
Chirp.objects.get(author__username='jeff')

MultipleObjectsReturned: get() returned more than one Chirp -- it returned 7!

# Limiting Results

In [17]:
Chirp.objects.all()[:2]

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: my first chirp by jeff posted my first chirp>]

In [22]:
Chirp.objects.all()[1:5]

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: Second by jeff posted Second>, <Chirp: Third by jeff posted Third>, <Chirp: Bob first by jeff posted Bob first>]

# Filters on Fields

In [28]:
Chirp.objects.filter(title__exact='my first chirp')

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: my first chirp by jeff posted my first chirp>]

In [29]:
Chirp.objects.filter(title__contains='first')

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: my first chirp by jeff posted my first chirp>, <Chirp: Bob first by jeff posted Bob first>, <Chirp: Actual Bob first by bob posted Actual Bob first>]

In [30]:
Chirp.objects.filter(title__startswith='my')

[<Chirp: my first chirp by jeff posted my first chirp>, <Chirp: my first chirp by jeff posted my first chirp>]

# Spanning Relationships

In [32]:
Chirp.objects.filter(author__username="bob")

[<Chirp: Actual Bob first by bob posted Actual Bob first>, <Chirp: Actual Bob second by bob posted Actual Bob second>]

#### It works in reverse as well...kinda

In [34]:
User.objects.filter(chirp__title='my first chirp')

[<User: jeff>, <User: jeff>]

Very important note.  The filter syntax doesn't use chirp_set it just uses the name of the model

In [41]:
bytes = b'{\xe2\x80\x9ccupsRemoved\xe2\x80\x9d : 3 }'

In [45]:
json_stuff = bytes.decode('Utf8')
json_stuff

'{“cupsRemoved” : 3 }'

In [46]:
json_stuff = '{"cupsRemoved" : 3 }'

In [43]:
import json

In [47]:
json.loads(json_stuff)

{'cupsRemoved': 3}

# F() ing objects

In [48]:
Chirp.objects.filter(message__icontains=F('author__username'))

[<Chirp: Actual Bob first by bob posted Actual Bob first>, <Chirp: Actual Bob second by bob posted Actual Bob second>]

# PK shortcut
pk is not actually a field

In [49]:
Chirp.objects.get(pk=1)

<Chirp: my first chirp by jeff posted my first chirp>

implies

In [50]:
Chirp.objects.get(id=1)

<Chirp: my first chirp by jeff posted my first chirp>

pk is just an alias for whatever the primary key is

#Q() ing Objects

Thus far everything we have done is basically an AND.  What if we wanted a couple of OR conditions?

In [51]:
from django.db.models import Q

#### Get all chirps that start with the word Second or Third

In [54]:
Chirp.objects.filter(Q(title__startswith='Second') | Q(title__startswith='Third'))

[<Chirp: Second by jeff posted Second>, <Chirp: Third by jeff posted Third>]

#### Get all chirps that start with the word Second or are not authored by jeff

In [55]:
Chirp.objects.filter(Q(title__startswith='Second') | ~Q(author__username='jeff'))

[<Chirp: Second by jeff posted Second>, <Chirp: Actual Bob first by bob posted Actual Bob first>, <Chirp: Actual Bob second by bob posted Actual Bob second>]

# Raw for when Django can't cut it

In [57]:
Chirp.objects.raw('''SELECT * FROM updates_chirp''')[0]

<Chirp: my first chirp by jeff posted my first chirp>

### Warning

Be __VERY__ careful to escape your input if a user has provided any of the filters.  Django does this for you automatically until you use ```.raw``` then you are on your own.

# Optimizing with select_related()

In [58]:
chirp = Chirp.objects.get(pk=1) #First database call

In [59]:
chirp.author #Another database query

<User: jeff>

### This is called an n+1 query problem

The system runs the main query to get the first object or list(+1) then each object executes another query to get the author (n).

### Let's optimize

In [ ]:
chirp = Chirp.objects.select_related()